In [ ]:
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
import numpy as np
from utils import listdir
import os

class PredictedVoxelDataset:
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.list_category = sorted(os.listdir('predicted_EVC_conv5'))
        voxel_path = []
        categories = []
        nSamples = []
        for i, c in enumerate(self.list_category):
            voxel_path.extend([os.path.join(self.root_dir, c, s) for s in os.listdir(os.path.join(self.root_dir, c))])
            nSample = len([s for s in os.listdir(os.path.join(self.root_dir, c))])
            if i == 0:
                nSamples.append(nSample)
            else:
                nSamples.append(nSample+nSamples[i-1])
            for n in range(nSample):
                categories.append(i)
        nSamples.insert(0, 0)
        self.voxel_path = voxel_path
        self.categories = categories
        self.nClass = len(set(categories))
        self.nSamples = nSamples
    def __len__(self):
        return len(self.voxel_path)
    def __getitem__(self, idx):
        #category = self.list_category[idx]
        voxels = np.load(self.voxel_path[idx])[0]
        #onehotclass = np.zeros(self.nClass)
        #onehotclass[self.categories[idx]] = 1
        sample = {'voxel': voxels, 'category': self.categories[idx]}
        return sample

In [ ]:
voxel_dataset = PredictedVoxelDataset(root_dir='./predictedvggall/subj001/LOC_conv5')
print(len(voxel_dataset))
print(voxel_dataset[0]['voxel'].shape)
print(voxel_dataset[0]['category'])

voxels = np.stack(voxel_dataset[i]['voxel'] for i in range(len(voxel_dataset)))
categories = np.stack(voxel_dataset[i]['category'] for i in range(len(voxel_dataset)))


In [ ]:
from tqdm import tqdm
classifier = svm.SVC(kernel='linear')

fmri_score = np.zeros((voxel_dataset.nClass, voxel_dataset.nClass))
for i in tqdm(range(0, voxel_dataset.nClass-1, 1)):
    if i == 100:
        break
    voxel_set1 = voxels[voxel_dataset.nSamples[i]:voxel_dataset.nSamples[i+1]]
    #print(voxel_set1.shape)
    category_set1 = categories[voxel_dataset.nSamples[i]:voxel_dataset.nSamples[i+1]]
    for j in range(i+1, voxel_dataset.nClass, 1):
        voxel_set2 = voxels[voxel_dataset.nSamples[j]:voxel_dataset.nSamples[j+1]]
        category_set2 = categories[voxel_dataset.nSamples[j]:voxel_dataset.nSamples[j+1]]
        #print(np.concatenate((voxel_set1, voxel_set2)))
        #print(np.concatenate((category_set1, category_set2)))
        X_train, X_test, y_train, y_test = train_test_split(
             np.concatenate((voxel_set1, voxel_set2)), np.concatenate((category_set1, category_set2)), test_size=0.5, shuffle=True)
        classifier.fit(X_train, y_train)
        predicted = classifier.predict(X_test)
        #print(metrics.classification_report(y_test, predicted))
        fmri_score[i,j-1] = classifier.score(X_test, y_test)

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(fmri_score[:100, :])
plt.show()
np.mean(fmri_score[:100, :])

In [ ]:
voxel_dataset = PredictedVoxelDataset(root_dir='./predictedvggall_random/subj001/LOC_conv5')
print(len(voxel_dataset))
print(voxel_dataset[0]['voxel'].shape)
print(voxel_dataset[0]['category'])

voxels = np.stack(voxel_dataset[i]['voxel'] for i in range(len(voxel_dataset)))
categories = np.stack(voxel_dataset[i]['category'] for i in range(len(voxel_dataset)))

In [ ]:
from tqdm import tqdm
classifier = svm.SVC(kernel='linear')

random_score = np.zeros((voxel_dataset.nClass, voxel_dataset.nClass))
for i in tqdm(range(0, voxel_dataset.nClass-1, 1)):
    if i == 100:
        break
    voxel_set1 = voxels[voxel_dataset.nSamples[i]:voxel_dataset.nSamples[i+1]]
    #print(voxel_set1.shape)
    category_set1 = categories[voxel_dataset.nSamples[i]:voxel_dataset.nSamples[i+1]]
    for j in range(i+1, voxel_dataset.nClass, 1):
        voxel_set2 = voxels[voxel_dataset.nSamples[j]:voxel_dataset.nSamples[j+1]]
        category_set2 = categories[voxel_dataset.nSamples[j]:voxel_dataset.nSamples[j+1]]
        #print(np.concatenate((voxel_set1, voxel_set2)))
        print(np.concatenate((category_set1, category_set2)))
        X_train, X_test, y_train, y_test = train_test_split(
             np.concatenate((voxel_set1, voxel_set2)), np.concatenate((category_set1, category_set2)), test_size=0.5, shuffle=True)
        classifier.fit(X_train, y_train)
        predicted = classifier.predict(X_test)
        #print(metrics.classification_report(y_test, predicted))
        random_score[i,j-1] = classifier.score(X_test, y_test)
        
    

In [ ]:
plt.figure(figsize=(10, 10))
plt.title('Mean classificaiton accuray (fMRI-derived subspace): ' + str(np.mean(fmri_score[:100, :])))
plt1 = plt.imshow(fmri_score[:100, :])
        plt.xticks(np.arange(nConds), conds, rotation=90)
        plt.yticks(np.arange(nConds), conds)
#print('Mean classificaiton accuray (fMRI-derived subspace): ', np.mean(fmri_score[:100, :]))
#plt.colorbar(plt1)

plt.figure(figsize=(10, 10))
plt.title('Mean classificaiton accuray (Random subspace): ' + str(np.mean(random_score[:100, :])))
plt2 = plt.imshow(random_score[:100, :])
plt.colorbar(plt2, orientation="horizontal")

#print('Mean classificaiton accuray (Random subspace): ', np.mean(random_score[:100, :]))

In [ ]:
# disp = metrics.plot_confusion_matrix(classifier, X_test, y_test)
# disp.figure_.suptitle("Confusion Matrix")
# print("Confusion matrix:\n%s" % disp.confusion_matrix)